In [1]:
### Imports
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import silhouette_score
from sklearn.linear_model import LinearRegression
from datetime import datetime
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler, Normalizer
from sklearn.neighbors import KNeighborsRegressor

In [2]:
### Load the dataset 
df = pd.read_csv('G:\Mi unidad\Hackathon--aws\Data\df_final.csv', compression='zip')
df.head()

,id_mutation,valeur_fonciere,surface_reelle_bati,nombre_pieces_principales,type_local,nature_mutation,nom_commune,code_commune,code_departement,id_parcelle,longitude,latitude,date_mutation
0,2019-1137255,310000.0,101.0,4.0,Maison,Vente,Gretz-Armainvilliers,77215,77,772150000B0473,2.740574,48.739669,2019-01-03
1,2019-1137256,150000.0,72.0,3.0,Maison,Vente,Saint-Fargeau-Ponthierry,77407,77,77407000AD0345,2.539824,48.563586,2019-01-03
2,2019-1137256,150000.0,72.0,3.0,Maison,Vente,Saint-Fargeau-Ponthierry,77407,77,77407000AD0345,2.539824,48.563586,2019-01-03
3,2019-1137257,250000.0,44.0,0.0,Local industriel. commercial ou assimilé,Vente,Melun,77288,77,77288000AV0266,2.656724,48.534489,2019-01-09
4,2019-1137258,155000.0,51.0,2.0,Appartement,Vente,Pontault-Combault,77373,77,77373000AD0157,2.616840,48.805639,2019-01-04


In [3]:
### Preprocessing -> Steps before training the ML model -> Get Dummies
df_dummies1 = pd.concat([df , df['type_local'].str.get_dummies()], 
          axis = 1)
df_final = pd.concat([df_dummies1 , df_dummies1['nature_mutation'].str.get_dummies()], 
          axis = 1)
df_final.head(4)

,id_mutation,valeur_fonciere,surface_reelle_bati,nombre_pieces_principales,type_local,nature_mutation,nom_commune,code_commune,code_departement,id_parcelle,...,date_mutation,Appartement,Local industriel. commercial ou assimilé,Maison,Adjudication,Echange,Expropriation,Vente,Vente en l'état futur d'achèvement,Vente terrain à bâtir
0,2019-1137255,310000.0,101.0,4.0,Maison,Vente,Gretz-Armainvilliers,77215,77,772150000B0473,...,2019-01-03,0,0,1,0,0,0,1,0,0
1,2019-1137256,150000.0,72.0,3.0,Maison,Vente,Saint-Fargeau-Ponthierry,77407,77,77407000AD0345,...,2019-01-03,0,0,1,0,0,0,1,0,0
2,2019-1137256,150000.0,72.0,3.0,Maison,Vente,Saint-Fargeau-Ponthierry,77407,77,77407000AD0345,...,2019-01-03,0,0,1,0,0,0,1,0,0
3,2019-1137257,250000.0,44.0,0.0,Local industriel. commercial ou assimilé,Vente,Melun,77288,77,77288000AV0266,...,2019-01-09,0,1,0,0,0,0,1,0,0


In [18]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 824305 entries, 0 to 824304
Data columns (total 22 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   id_mutation                               824305 non-null  object 
 1   valeur_fonciere                           817036 non-null  float64
 2   surface_reelle_bati                       824305 non-null  float64
 3   nombre_pieces_principales                 824305 non-null  float64
 4   type_local                                824305 non-null  object 
 5   nature_mutation                           824305 non-null  object 
 6   nom_commune                               824305 non-null  object 
 7   code_commune                              824305 non-null  int64  
 8   code_departement                          824305 non-null  int64  
 9   id_parcelle                               824305 non-null  object 
 10  longitude           

# Neighborhood

In [4]:
### Nearest Neighbor - ML model - silhouette_score
start_time = datetime.now()

# Drop NaN
dft =  df_final.dropna()

# Characteristics for the model 
characteristics = ['valeur_fonciere','surface_reelle_bati','nombre_pieces_principales','longitude','latitude',
'Appartement','Local industriel. commercial ou assimilé','Maison','Adjudication','Echange','Expropriation','Vente',"Vente en l'état futur d'achèvement",
'Vente terrain à bâtir']

# Split the data into Train and test set
df_train, df_test = train_test_split(dft, test_size=0.2, random_state=7)

# Select the model and prepare Grid Search
model = NearestNeighbors()

parameters = {'n_neighbors':[3,4,5], 
                'algorithm':['ball_tree', 'kd_tree', 'brute'], 
                'metric': ['cityblock','euclidean','manhattan']
}

# GridSearch and fit in train
grid_search = GridSearchCV(model, parameters, cv=5, scoring = silhouette_score)
grid_search.fit(df_train[characteristics])

params_to_use = grid_search.best_params_
estimator = grid_search.best_estimator_

end_time = datetime.now()

#Check results
print(f'NaN values dropped: ', df_final.shape[0]- dft.shape[0])
print(f'Best score: ', grid_search.best_score_)
print(f'Best params: ', grid_search.best_params_)
print(f'Best estimator: ', grid_search.best_estimator_)
print('\nBest params: ', params_to_use)
print(f'\nTotalTime: ',(end_time - start_time))

c:\Users\frans\anaconda3\envs\Data\lib\site-packages\sklearn\model_selection\_validation.py:776: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\frans\anaconda3\envs\Data\lib\site-packages\sklearn\model_selection\_validation.py", line 765, in _score
    scores = scorer(estimator, X_test)
  File "c:\Users\frans\anaconda3\envs\Data\lib\site-packages\sklearn\metrics\cluster\_unsupervised.py", line 117, in silhouette_score
    return np.mean(silhouette_samples(X, labels, metric=metric, **kwds))
  File "c:\Users\frans\anaconda3\envs\Data\lib\site-packages\sklearn\metrics\cluster\_unsupervised.py", line 212, in silhouette_samples
    X, labels = check_X_y(X, labels, accept_sparse=["csc", "csr"])
  File "c:\Users\frans\anaconda3\envs\Data\lib\site-packages\sklearn\utils\validation.py", line 1074, in check_X_y
    X = check_array(
  File "c:\Users\frans\anaconda3\envs\Data

NaN values dropped:  15229
Best score:  nan
Best params:  {'algorithm': 'ball_tree', 'metric': 'cityblock', 'n_neighbors': 3}
Best estimator:  NearestNeighbors(algorithm='ball_tree', metric='cityblock', n_neighbors=3)

Best params:  {'algorithm': 'ball_tree', 'metric': 'cityblock', 'n_neighbors': 3}

TotalTime:  0:08:01.417516


In [8]:
def recommend_neighborhoods(current_neighborhood):
    distances, index1 = estimator.kneighbors(df_train[df_train['nom_commune'] == current_neighborhood][characteristics])
    similar_neighborhoods = df_train.iloc[index1.flat]['nom_commune']
    similar_neighborhoods = similar_neighborhoods[similar_neighborhoods != current_neighborhood].head(3)
    return similar_neighborhoods.values[0], similar_neighborhoods.values[1], similar_neighborhoods.values[2]
recommend_neighborhoods('Paris 14e Arrondissement')

('Montreuil', 'Paris 1er Arrondissement', 'Paris 15e Arrondissement')

In [12]:
### Main Function
# Drop NaN
dft =  df_final.dropna()

# Characteristics for the model 
characteristics = ['valeur_fonciere','surface_reelle_bati','nombre_pieces_principales','longitude','latitude',
'Appartement','Local industriel. commercial ou assimilé','Maison','Adjudication','Echange','Expropriation','Vente',"Vente en l'état futur d'achèvement",
'Vente terrain à bâtir']

# Split the data into Train and test set
df_train, df_test = train_test_split(dft, test_size=0.2, random_state=7)

# Select the model and prepare Grid Search
parameters = {'n_neighbors':3, 
                'algorithm':'ball_tree', 
                'metric': 'cityblock'
}
model = NearestNeighbors(n_neighbors = 3, algorithm = 'ball_tree', metric ='cityblock'). fit(df_train[characteristics])


def recommend_neighborhoods(current_neighborhood):
    distance, index1 = model.kneighbors(df_train[df_train['nom_commune'] == current_neighborhood][characteristics])
    similar_neighborhoods = df_train.iloc[index1.flat]['nom_commune']
    similar_neighborhoods = similar_neighborhoods[similar_neighborhoods != current_neighborhood].head(3)
    return similar_neighborhoods.values[0], similar_neighborhoods.values[1], similar_neighborhoods.values[2]
recommend_neighborhoods('Paris 14e Arrondissement')

('Montreuil', 'Paris 1er Arrondissement', 'Paris 15e Arrondissement')

# Price

In [4]:
dft =  df_final.dropna()

In [24]:
# Define target
dft1 = dft[dft['nom_commune']=='Montreuil']

X = dft1[['surface_reelle_bati','nombre_pieces_principales','Maison','Appartement']]
y = dft1['valeur_fonciere']

# Train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

# Model 
model01 = LinearRegression().fit(X_train,y_train)

print('mean square error: ',mean_squared_error(y_test, model01.predict(X_test)))
print('r2: ',r2_score(y_test, model01.predict(X_test)))

def prediction_price(sqm, rooms,house,apt): 
    user_input = {'surface_reelle_bati':[sqm],
                'nombre_pieces_principales': [rooms],
                'Maison':[house],
                'Appartement':[apt]}
    user_input_df = pd.DataFrame(user_input)
    price_pred = model01.predict(user_input_df)
    return price_pred

prediction_price(50,2,1,0)


-0.0786671722192962
mean square error:  142808171768392.62
r2:  -0.0786671722192962


array([-232721.93186432])

In [29]:
### Neighbours 
# Drop NaN
dft =  df_final.dropna()

# Characteristics for the model 
characteristics = ['valeur_fonciere','surface_reelle_bati','nombre_pieces_principales','longitude','latitude',
'Appartement','Local industriel. commercial ou assimilé','Maison','Adjudication','Echange','Expropriation','Vente',"Vente en l'état futur d'achèvement",
'Vente terrain à bâtir']

# Split the data into Train and test set
df_train, df_test = train_test_split(dft, test_size=0.2, random_state=7)

# Select the model and prepare Grid Search
parameters = {'n_neighbors':3, 
                'algorithm':'ball_tree', 
                'metric': 'cityblock'
}
model = NearestNeighbors(n_neighbors = 3, algorithm = 'ball_tree', metric ='cityblock'). fit(df_train[characteristics])

In [42]:

def recommend_neighborhoods(current_neighborhood):
    distance, index1 = model.kneighbors(df_train[df_train['nom_commune'] == current_neighborhood][characteristics])
    similar_neighborhoods = df_train.iloc[index1.flat]['nom_commune']
    similar_neighborhoods = similar_neighborhoods[similar_neighborhoods != current_neighborhood].head(3)
    return similar_neighborhoods

district = recommend_neighborhoods('Paris 1er Arrondissement')
print(district.values)

dft1 = dft[dft['nom_commune']== district.values[0]]

X = dft1[['surface_reelle_bati','nombre_pieces_principales','Maison','Appartement']]
y = dft1['valeur_fonciere']

# Train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Prepare the model
model01 = LinearRegression()
param_grid = {'normalize':[True, False]}
grid_search = GridSearchCV(model01, param_grid, cv=5, scoring='r2')

# Fit the grid search object to the data
grid_search.fit(X_train,y_train)

def prediction_price(sqm, rooms,house,apt): 
    user_input = {'surface_reelle_bati':[sqm],
                'nombre_pieces_principales': [rooms],
                'Maison':[house],
                'Appartement':[apt]}
    user_input_df = pd.DataFrame(user_input)
    price_pred = grid_search.best_estimator_.predict(user_input_df)
    return price_pred

prediction_price(100,2,0,1)
print(prediction_price(100,2,0,1))
print("Best score: ", grid_search.best_score_)

['Paris 18e Arrondissement' 'Paris 15e Arrondissement' 'Croissy-sur-Seine']
[64736777.46487492]
Best score:  0.05042090308995806


c:\Users\frans\anaconda3\envs\Data\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
c:\Users\frans\anaconda3\envs\Data\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import ma

In [35]:
dft1 = dft[dft['nom_commune']== 'Paris 15e Arrondissement']

X = dft1[['surface_reelle_bati','nombre_pieces_principales','Maison','Appartement']]
y = dft1['valeur_fonciere']

# Scale input features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the hyperparameter grid
param_grid = {'normalize':[True, False]}

# Create the grid search object
grid_search = GridSearchCV(model01, param_grid, cv=5, scoring='r2')

# Fit the grid search object to the data
grid_search.fit(X_train,y_train)

# Print the best parameters and best score
print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Best parameters:  {'normalize': False}
Best score:  0.05042090308995806


c:\Users\frans\anaconda3\envs\Data\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), LinearRegression())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  warnings.warn(
c:\Users\frans\anaconda3\envs\Data\lib\site-packages\sklearn\linear_model\_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import ma

In [43]:
dft.info

<bound method DataFrame.info of          id_mutation  valeur_fonciere  surface_reelle_bati  \
0       2019-1137255         310000.0                101.0   
1       2019-1137256         150000.0                 72.0   
2       2019-1137256         150000.0                 72.0   
3       2019-1137257         250000.0                 44.0   
4       2019-1137258         155000.0                 51.0   
...              ...              ...                  ...   
824300   2022-537268        1905000.0                160.0   
824301   2022-537269         270000.0                 51.0   
824302   2022-537270        1326666.6                281.0   
824303   2022-537271         423000.0                 43.0   
824304   2022-537272         104650.0                 15.0   

        nombre_pieces_principales                                type_local  \
0                             4.0                                    Maison   
1                             3.0                                

In [47]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

dft1 = dft[dft['nom_commune']== 'Paris 15e Arrondissement']

X = dft1[['surface_reelle_bati','nombre_pieces_principales','type_local', 'nature_mutation']]
y = dft1['valeur_fonciere']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Encode categorical variables
categorical_cols = ['type_local', 'nature_mutation']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_cols)])

# create linear regression model
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('regressor', LinearRegression())])

# fit the model to training data
model.fit(X_train, y_train)

# predict prices for test data
y_pred = model.predict(X_test)
print( model.score(X_test,y_test))

0.011371849854552418


In [ ]:
i